In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
test_err = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_test_quality.csv', parse_dates=['time'])
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case1.csv')

In [8]:
df1 = test_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1


In [9]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1
...,...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0,1
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1,1


In [10]:
# multiple_model
model_df =  test_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
30000,1
30001,0
30002,0
30003,0
30004,1
30005,0
30006,0


In [12]:
# multiple_fwver
fwver_df =  test_err[['user_id', 'fwver']]
fwver_df = fwver_df.drop_duplicates(subset=None, keep='first')

fwver_num = fwver_df.user_id.value_counts(sort=False)
user_has_multiple_fwvers = fwver_num > 1

user_has_multiple_fwvers = user_has_multiple_fwvers.astype(int)
user_has_multiple_fwvers = user_has_multiple_fwvers.to_frame(name='multiple_fwver')
user_has_multiple_fwvers.head(7)

,multiple_fwver
30000,1
30001,0
30002,1
30003,1
30004,1
30005,0
30006,0
